In [70]:
import pickle
import pandas as pd
from _html_parser import HtmlParser, ParsingDataPrepare, TableConstant

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)

In [2]:
%load_ext autoreload
%autoreload 2

### Competitions tables parser

In [55]:
# df_26_05_2022_to_27_03_2021_games, df_26_03_2021_to_03_11_2013_games, df_02_11_2013_to_10_07_2005_games, df_09_07_2005_to_01_01_2000_games
# df_29_12_1999_to_15_08_1998_games
with open('pickle_files/new_events/29_12_1999/df_29_12_1999_to_15_08_1998_games', 'rb') as f:
    df_games = pickle.load(f)

In [56]:
df_games

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue
0,10742693,Premier League,12,43,29.12.1999 19:00,Chelsea - Sheffield Wednesday,3:0,top
1,10742692,Premier League,12,43,29.12.1999 19:00,Aston Villa - Tottenham,1:1,top
2,10742689,Premier League,12,43,28.12.1999 14:00,Sunderland - Manchester United,2:2,top
3,10742685,Premier League,12,43,28.12.1999 14:00,Arsenal - Leeds United,2:0,top
4,10742688,Premier League,12,43,28.12.1999 14:00,Liverpool - Wimbledon,3:1,top
...,...,...,...,...,...,...,...,...
2073,11385916,2. Bundesliga,587,44,16.08.1998 13:00,Köln - Ulm 1846,1:1,top
2074,11385917,2. Bundesliga,587,44,16.08.1998 13:00,Stuttgarter Kickers - Fortuna Düsseldorf,1:0,top
2075,11385915,2. Bundesliga,587,44,16.08.1998 13:00,Greuther Fürth - Rot-Weiß,1:1,top
2076,11385919,2. Bundesliga,587,44,16.08.1998 13:00,Wattenscheid 09 - Energie Cottbus,1:1,top


In [57]:
comp_seasons = ParsingDataPrepare.competition_season_ids(df_games)

In [58]:
list(comp_seasons.keys())[3], list(comp_seasons.values())[3]

('19', [])

In [59]:
len(comp_seasons)

22

In [60]:
dict_seasons = dict((x, y) for x, y in comp_seasons.items() if y != [])

In [62]:
# dict_seasons

In [63]:
list(dict_seasons.keys())[2], list(dict_seasons.values())[2] 

('17', ['43', '44'])

In [64]:
comp_tables = HtmlParser.parsing_competition_tables(dict_seasons)

12 43
12 44
1253 25
17 43
17 44
587 43
587 44
617 31
618 31
837 31


In [65]:
len(comp_tables)

7

In [67]:
# comp_tables

In [68]:
df_comp_tables = ParsingDataPrepare.create_df_comp_tables(comp_tables)

In [71]:
df_comp_tables

,team_name,team_id,pos,pld,w,d,l,gf,ga,+/-,pts,comp_id,season_id
0,Man Utd,92,1,38,28,7,3,97,45,+52,91,12,43
1,Arsenal,2,2,38,22,7,9,73,43,+30,73,12,43
2,Leeds United,6930,3,38,21,6,11,58,43,+15,69,12,43
3,Liverpool,79,4,38,19,10,9,51,30,+21,67,12,43
4,Chelsea,184,5,38,18,11,9,53,34,+19,65,12,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Dinamo MVD,18604,4,3,0,0,3,1,7,-6,0,837,31
193,Spartak Moscow II,7511,1,3,3,0,0,5,1,+4,9,837,31
194,Spartak Erevan,10442,2,3,2,0,1,4,3,+1,6,837,31
195,Spartak Semey,8271,3,3,0,1,2,3,5,-2,1,837,31


#### Fix and clear data incorrect

In [72]:
df_comp_tables.pos.value_counts(dropna=False)

1     26
4     25
3     23
2     17
5     16
      ..
13     6
12     6
10     6
19     2
20     2
Name: pos, Length: 20, dtype: int64

In [29]:
# df_comp_tables[df_comp_tables.pos == '364']

In [74]:
# file = open('pickle_files/new_events/df_comp_tabs_29_12_1999_all_lgs', 'wb')
# pickle.dump(df_comp_tables, file)  
# file.close()

In [76]:
with open('pickle_files/new_events/df_comp_tabs_29_12_1999_all_lgs', 'rb') as f:
    df_comp_tabs = pickle.load(f)

In [77]:
df_comp_tabs

,team_name,team_id,pos,pld,w,d,l,gf,ga,+/-,pts,comp_id,season_id
0,Man Utd,92,1,38,28,7,3,97,45,+52,91,12,43
1,Arsenal,2,2,38,22,7,9,73,43,+30,73,12,43
2,Leeds United,6930,3,38,21,6,11,58,43,+15,69,12,43
3,Liverpool,79,4,38,19,10,9,51,30,+21,67,12,43
4,Chelsea,184,5,38,18,11,9,53,34,+19,65,12,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Dinamo MVD,18604,4,3,0,0,3,1,7,-6,0,837,31
193,Spartak Moscow II,7511,1,3,3,0,0,5,1,+4,9,837,31
194,Spartak Erevan,10442,2,3,2,0,1,4,3,+1,6,837,31
195,Spartak Semey,8271,3,3,0,1,2,3,5,-2,1,837,31
